In [46]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.language.questionanswering import QuestionAnsweringClient

endpoint = "https://question-answering-test-simplon.cognitiveservices.azure.com/"
credential = AzureKeyCredential("e518b6bec9204e41b9cf4b968ddc83f6")
knowledge_base_project = "Projet-X"
deployment = "production"

def main():
    client = QuestionAnsweringClient(endpoint, credential)
    with client:
        question="How much battery life do I have left?"
        output = client.get_answers(
            question = question,
            project_name=knowledge_base_project,
            deployment_name=deployment
        )
    print("Q: {}".format(question))
    print("A: {}".format(output.answers[0].answer))

if __name__ == '__main__':
    main()

Q: How much battery life do I have left?
A: No answer found


In [3]:
f = open('/home/apprenant/Bureau/NLP_QA_Project/Dataset/43_Arya_Stark.txt')
text =f.read()[:5000]

In [9]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.language.questionanswering import QuestionAnsweringClient
from azure.ai.language.questionanswering import models as qna
from dotenv import load_dotenv

load_dotenv()
endpoint = "https://question-answering-test-simplon.cognitiveservices.azure.com/"
credential = AzureKeyCredential(os.getenv('credential'))
def get_answer(text,question):
    client = QuestionAnsweringClient(endpoint, credential)
    with client:
        input = qna.AnswersFromTextOptions(
            question=question,
            text_documents=[text]
        )


        output = client.get_answers_from_text(input)

    best_answer = [a for a in output.answers if a.confidence > 0.5][0]
    print(u"Q: {}".format(input.question))
    print(u"A: {}".format(best_answer.answer))
    print("Confidence Score: {}".format(output.answers[0].confidence))

get_answer(bladjakjdazkljdklazjdklazjdlazjdkl love is life'', ' What is love ?')


Q:  What is love ?
A: bladjakjdazkljdklazjdklazjdlazjdkl,
           love is life
Confidence Score: 0.21424047648906708


In [25]:
import PyPDF2
 
#create file object variable
#opening method will be rb
file=open('/home/apprenant/Bureau/NLP_QA_Project/Dataset/lab_eval.pdf','rb')
 
#create reader variable that will read the pdffileobj
reader = PyPDF2.PdfReader(file)
liste = []
#This will store the number of pages of this pdf file
x= len(reader.pages)
print(x)
for i in range(x) :
    page = reader.pages[i]
    text = page.extract_text()
    liste.append(text)

file1=open(r"/home/apprenant/Bureau/NLP_QA_Project/Dataset/1.txt","a")
file1.writelines(liste)

5


12

In [ ]:
from tqdm.auto import tqdm
import pinecone
import torch
from sentence_transformers import SentenceTransformer
# connect to pinecone environment
def initilisation():
    pinecone.init(
        api_key="1b8c3b2a-c0a7-4ed1-83fc-0ae03d406550",
        environment="us-central1-gcp"
    )
    index_name = "test"

    # check if the extractive-question-answering index exists
    if index_name not in pinecone.list_indexes():
        # create the index if it does not exist
        pinecone.create_index(
            index_name,
            dimension=384,
            metric="cosine"
        )

    # connect to extractive-question-answering index we created
    # set device to GPU if available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    # load the retriever model from huggingface model hub
    retriever = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1', device=device)
    retriever
    
    dico = {}
    dico['index']= pinecone.Index(index_name)
    dico['retriever'] = retriever
    dico['device']

    return dico

# we will use batches of 64
def save_document(df,retriever,index):
    batch_size = 64
    for i in tqdm(range(0, len(df), batch_size)):
        # find end of batch
        i_end = min(i+batch_size, len(df))
        # extract batch
        batch = df.iloc[i:i_end]
        # generate embeddings for batch
        emb = retriever.encode(batch["context"].tolist()).tolist()
        # get metadata
        meta = batch.to_dict(orient="records")
        # create unique IDs
        ids = [f"{idx}" for idx in range(i, i_end)]
        # add all to upsert list
        to_upsert = list(zip(ids, emb, meta))
        # upsert/insert these records to pinecone
        _ = index.upsert(vectors=to_upsert)
        
from transformers import pipeline
from pprint import pprint

model_name = "deepset/electra-base-squad2"
# load the reader model into a question-answering pipeline
reader = pipeline(tokenizer=model_name, model=model_name, task="question-answering", device=device)
def get_context(question, top_k):
    # generate embeddings for the question
    xq = retriever.encode([question]).tolist()
    # search pinecone index for context passage with the answer
    xc = index.query(xq, top_k=top_k, include_metadata=True)
    # extract the context passage from pinecone search result
    c = [x["metadata"]["context"] for x in xc["matches"]]
    return c

def extract_answer(question, context):
    results = []
    for c in context:
        # feed the reader the question and contexts to extract answers
        answer = reader(question=question, context=c)
        # add the context to answer dict for printing both together
        answer["context"] = c
        results.append(answer)
    # sort the result based on the score from reader model
    sorted_result = pprint(sorted(results, key=lambda x: x["score"], reverse=True))
    return sorted_result

In [109]:
from PyPDF2 import PdfReader
import pdfrw
import re
import pandas as pd
reader = PdfReader('/home/apprenant/Bureau/NLP_QA_Project/Dataset/Brief - Agile - NLP.pdf')

title_list = []
context_list = []
text = ''
clean_text = ''
for i, page in enumerate(reader.pages):                
    text += f'\n\nPage n°{i+1} :\n'
    n =page.extract_text()
    text += n
clean_text = text.replace(" ","").replace('●', '').replace("\n"," ")
text= text.split('\n\n')

for i in range(1,len(text)):
    text[i]=text[i].replace(" ","").replace('●', '').replace("\n"," ")
    context_list.append(text[i])
    title_list.append('test')


df =pd.DataFrame({"titre":title_list,"contexte":context_list})
clean_text

"  Pagen°1: Brief Agile NLP Vous travaillez dans une agence digitale. Votre principal client est un groupe bancaire. Ce dernier aimerait explorer les usages innovants liés au NLP et vous demande de faire une étude de faisabilité sur ce sujet. Vous allez devoir réaliser une veille technologique ainsi qu’une preuve de concept liée au secteur bancaire. Votre mission :  Expliquer clairement la tâche de NLP sur laquelle vous travaillez  Faire une veille technique sur les différents outils et solutions sur le marché pour répondre à cette tâche  Faire un POC mis en production sur un sujet de votre choix lié au secteur bancaire qui permettra d’implémenter cette tâche.  Utiliser la démarche Agile tout au long du projet Les tâches : 1. Question Answering [Jeffrey, Anas, Benjamin] 2. Entity Recognition [Yanis, Emad, Sarah, Hamza] 3. Conversational (chat/bot) [Cédric, Omar, Kamel, Kevin] 4. Text Classification [Léon, Zouhair, Abdellah, Patrick] 5. Summarization [Thomas, Adeline, Steven, Elias] 1 -

In [95]:
text

['',
 'Pagen°1: Brief Agile NLP Vous travaillez dans une agence digitale. Votre principal client est un groupe bancaire. Ce dernier aimerait explorer les usages innovants liés au NLP et vous demande de faire une étude de faisabilité sur ce sujet. Vous allez devoir réaliser une veille technologique ainsi qu’une preuve de concept liée au secteur bancaire. Votre mission :',
 'Expliquer clairement la tâche de NLP sur laquelle vous travaillez',
 'Faire une veille technique sur les différents outils et solutions sur le marché pour répondre à cette tâche',
 'Faire un POC mis en production sur un sujet de votre choix lié au secteur bancaire qui permettra d’implémenter cette tâche.',
 'Utiliser la démarche Agile tout au long du projet Les tâches : 1. Question Answering [Jeffrey, Anas, Benjamin] 2. Entity Recognition [Yanis, Emad, Sarah, Hamza] 3. Conversational (chat/bot) [Cédric, Omar, Kamel, Kevin] 4. Text Classification [Léon, Zouhair, Abdellah, Patrick] 5. Summarization [Thomas, Adeline, St

In [94]:
df

,titre,contexte
0,test,Pagen°1: Brief Agile NLP Vous travaillez dans ...
1,test,Expliquer clairement la tâche de NLP sur laque...
2,test,Faire une veille technique sur les différents ...
3,test,Faire un POC mis en production sur un sujet de...
4,test,Utiliser la démarche Agile tout au long du pro...
5,test,Un modèle Hugging Face entraîné avec Tensor Fl...
6,test,L’utilisation d’un service Azure Cognitive Ser...
7,test,Pagen°2: L’application web ainsi que les modèl...
8,test,Mettre en place la méthode Agile dans un proje...
9,test,ce qui a été fait hier


In [84]:
new = ''
for i in range(1,len(text)):
    text[i]=text[i].replace('\n','').replace('●', '').
new = text[1:5]
new

['Pagen°1:BriefAgileNLPVoustravaillezdansuneagencedigitale.Votreprincipalclientestungroupebancaire.CedernieraimeraitexplorerlesusagesinnovantsliésauNLPetvousdemandedefaireuneétudedefaisabilitésurcesujet.Vousallezdevoirréaliseruneveilletechnologiqueainsiqu’unepreuvedeconceptliéeausecteurbancaire.Votremission:ExpliquerclairementlatâchedeNLPsurlaquellevoustravaillezFaireuneveilletechniquesurlesdifférentsoutilsetsolutionssurlemarchépourrépondreàcettetâcheFaireunPOCmisenproductionsurunsujetdevotrechoixliéausecteurbancairequipermettrad’implémentercettetâche.UtiliserladémarcheAgiletoutaulongduprojetLestâches:1.QuestionAnswering[Jeffrey,Anas,Benjamin]2.EntityRecognition[Yanis,Emad,Sarah,Hamza]3.Conversational(chat/bot)[Cédric,Omar,Kamel,Kevin]4.TextClassification[Léon,Zouhair,Abdellah,Patrick]5.Summarization[Thomas,Adeline,Steven,Elias]1-ExplicationdelatâchedeNLPSitouteslestâchesmettentenœuvreunmodèleprédictif,ellessonttoutesdenaturetrèsdifférentes.Ilfaudradoncexpliquerlesconceptsclésquilesent

In [38]:
new =

for i in range(1,len(text)) :
    text[i].replace('\n', ' ')

text

' r i e fA g i l eN L PV o u st r a v a i l l e zd a n su n ea g e n c ed i g i t a l e .V o t r ep r i n c i p a lc l i e n te s tu ng r o u p eb a n c a i r e .C ed e r n i e ra i m e r a i te x p l o r e rl e su s a g e si n n o v a n t sl i é sa uN L Pe tv o u sd e m a n d ed ef a i r eu n eé t u d ed ef a i s a b i l i t és u rc es u j e t .V o u sa l l e zd e v o i rr é a l i s e ru n ev e i l l et e c h n o l o g i q u ea i n s iq u ’ u n ep r e u v ed ec o n c e p tl i é ea us e c t e u rb a n c a i r e .V o t r em i s s i o n:●E x p l i q u e rc l a i r e m e n tl at â c h ed eN L Ps u rl a q u e l l ev o u st r a v a i l l e z●F a i r eu n ev e i l l et e c h n i q u es u rl e sd i f f é r e n t so u t i l se ts o l u t i o n ss u rl em a r c h ép o u rr é p o n d r eàc e t t et â c h e●F a i r eu nP O Cm i se np r o d u c t i o ns u ru ns u j e td ev o t r ec h o i xl i éa us e c t e u rb a n c a i r eq u ip e r m e t t r ad ’ i m p l é m e n t e rc e t t et â c h e .●U t i 

In [34]:
import torch
torch.cuda.current_device()


0

In [35]:
torch.cuda.is_available()

True

In [37]:
torch.device("cuda")

device(type='cuda')